In [1]:
import torch
import numpy as np
import cv2
import time
import datetime
import pandas as pd
from PIL import Image
from yolov5.models.experimental import attempt_load

from convert_data import collect_data
from class_mapping import yolo_class_mapping, fire_class_mapping, animals_class_mapping
from detection import category
from perform_object import perform_object
from yolo_to_server import send_server

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



#### Version check

In [2]:
print('opencv version: ', cv2.__version__)

opencv version:  4.8.0


In [3]:
camera_name = 'KFQ StarValley'
device = torch.device('cpu')

# Load trained YOLOv5 model weights
yolo_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/yolov5m.pt'
fire_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/fire_smoke.pt'
animals_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/animals.pt'

yolo_model = attempt_load(yolo_weights_path, device=device)
fire_model = attempt_load(fire_weights_path, device=device)
animals_model = attempt_load(animals_weights_path, device=device)

# Initialize webcam
cap = cv2.VideoCapture(0)

# Create an empty DataFrame to store results
columns = ['Time', 'Class_Name', 'Confidence', 'X1', 'Y1', 'X2', 'Y2', 'action_detection', 'action_category', 'event_type']
results_df = pd.DataFrame(columns=columns)

while True:
    ret, frame = cap.read()  # Read frame from the webcam
    captured_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 상황 발생 시간 기록
    
    # Preprocess the frame
    image = Image.fromarray(frame)
    image = image.resize((640, 640))  # Resize the image to the model's input size
    image_tensor = torch.from_numpy(np.array(image)).float().permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    # Perform object detection
    results = perform_object(image_tensor, 0.65, yolo_model)
    results_df = collect_data(results, frame, yolo_class_mapping, results_df, captured_time)
    print(results_df)
    
    results = perform_object(image_tensor, 0.3, fire_model)
    results_df = collect_data(results, frame, fire_class_mapping, results_df, captured_time)
    #print(results_df)j
    
    results = perform_object(image_tensor, 0.65, animals_model)
    results_df = collect_data(results, frame, animals_class_mapping, results_df, captured_time)
    #print(results_df)
    
    results_df = category(results_df)

    # Display the frame with bounding boxes and labels
    cv2.imshow(camera_name, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit the loop
        break
    
    # send_server(results_df, camera_name)
    
    time.sleep(2) # 2초 간격으로 사진 캡쳐(30 frames per 1 miniute)

# Save the DataFrame to a CSV file
results_df.to_csv('object_detection_results.csv', index=False)

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Fusing layers... 
YOLOv5m summary: 369 layers, 21190557 parameters, 0 gradients
Fusing layers... 
YOLOv5m summary: 369 layers, 20879400 parameters, 0 gradients, 48.2 GFLOPs
Fusing layers... 
YOLOv5m summary: 369 layers, 20887482 parameters, 0 gradients, 48.3 GFLOPs


Empty DataFrame
Columns: [Time, Class_Name, Confidence, X1, Y1, X2, Y2, action_detection, action_category, event_type]
Index: []
                  Time Class_Name       Confidence    X1   Y1    X2   Y2  \
0  2023-08-18 14:25:21       fire  tensor(0.53789)  1320  652  1849  809   
1  2023-08-18 14:25:21       fire  tensor(0.38535)  1333  807  1646  955   

   action_detection action_category event_type  
0                 1   fire detected        F01  
1                 1   fire detected        F01  
                  Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0  2023-08-18 14:25:21       fire  tensor(0.53789)  1320  652  1849   809   
1  2023-08-18 14:25:21       fire  tensor(0.38535)  1333  807  1646   955   
2  2023-08-18 14:25:24       fire  tensor(0.88432)  1290  604  1932  1065   

   action_detection action_category event_type  
0                 1   fire detected        F01  
1                 1   fire detected        F01  
2                 1   fire detected    

2023-08-18 14:25:35.370 python[62046:3880338] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (2.55 secs).


                  Time Class_Name       Confidence    X1   Y1    X2    Y2  \
0  2023-08-18 14:25:21       fire  tensor(0.53789)  1320  652  1849   809   
1  2023-08-18 14:25:21       fire  tensor(0.38535)  1333  807  1646   955   
2  2023-08-18 14:25:24       fire  tensor(0.88432)  1290  604  1932  1065   
3  2023-08-18 14:25:27       fire  tensor(0.88644)  1292  602  1931  1064   
4  2023-08-18 14:25:29       fire  tensor(0.88407)  1292  602  1932  1064   
5  2023-08-18 14:25:32       fire  tensor(0.88222)  1292  601  1933  1064   
6  2023-08-18 14:25:34     person  tensor(0.74290)  1402  443  1477   570   
7  2023-08-18 14:25:34       fire  tensor(0.88707)  1291  602  1930  1065   
8  2023-08-18 14:25:37        bus  tensor(0.93778)  1058  354  1911   738   
9  2023-08-18 14:25:37        bus  tensor(0.78485)  1220  511  1916  1078   

   action_detection  action_category event_type  
0                 1    fire detected        F01  
1                 1    fire detected        F01  
2 